# Agrupamento de features categóricas (Modelo de classificação) - Implantação

Este componente realiza o agrupamento de features categóricas com alta cardinalidade para serem implementadas por modelos de classificação. Neste componente são implementadas três estratégias de agrupamento (method):

* percent: o agrupamento é feito considerando um ponto de corte definido. Categorias abaixo desse limiar são agrupadas em uma categoria denominada "other";

* top_n: apenas as n categorias mais frequentes são mantidas. As demais são agrupadas em uma categoria denominada "other";

* kmeans: Para cada classe, são calculadas sua frequência relativa e porcentagem de casos positivos da variável resposta. Então, o algoritmo kmeans utiliza estas informações para o agrupamento. 

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import numpy as np
import pandas as pd


class ApplyGroupCatFeatures:
    def __init__(self, columns_changed=None, columns=None, dict_class={}):
        # Input variables
        self.columns_changed = columns_changed
        self.dict_class = dict_class
        self.columns = columns

    def apply_group(self, X):
        cat_columns = self.columns_changed
        X_cat = X[cat_columns]

        for col in cat_columns:
            X_cat.loc[:, col] = self.dict_class[col + "....grouped"][
                self.dict_class[col].index(X.loc[:, col].to_numpy())
            ]

        columns_tocopy = np.setdiff1d(self.columns, cat_columns)

        X_res = pd.concat([X_cat, X[columns_tocopy]], axis=1)

        return X_res


class Model(object):
    def __init__(self):
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/grouping-categorical-features.joblib")
        self.columns = artifacts["columns"]
        self.columns_changed = artifacts["columns_changed"]
        self.features_after_pipeline = artifacts["features_after_pipeline"]
        self.dict_class = artifacts["dict_class"]

    def class_names(self):
        return self.features_after_pipeline.tolist()

    def predict(self, X, feature_names, meta=None):
        # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
        X = pd.DataFrame(X, columns=feature_names)

        g = ApplyGroupCatFeatures(
            columns_changed=self.columns_changed,
            columns=self.columns,
            dict_class=self.dict_class,
        )
        # Realiza a transformação
        X_res = g.apply_group(X)

        return X_res.to_numpy()

## Teste do serviço REST

Crie um arquivo `contract.json` com os seguintes atributos:

- `features` : A lista de features em uma requisição.
- `targets` : A lista de valores retornados pelo método `predict`.

Cada `feature` pode conter as seguintes informações:

- `name` : nome da feature
- `ftype` : tipo da feature : **continuous** ou **categorical**
- `dtype` : tipo de dado : **FLOAT** ou **INT** : *obrigatório para ftype continuous*
- `range` : intervalo de valores numéricos : *obrigatório para ftype continuous*
- `values` : lista de valores categóricos : *obrigatório para ftype categorical*

Em seguida, utilize a função `test_deployment` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para simular predição em tempo-real.<br>

In [ ]:
%%writefile contract.json
{
    "features": [
        {
            "name": "Abbrev",
            "dtype": "INT",
            "ftype": "continuous",
            "range": [1, 15]
        },
        {
            "name": "Rep",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1.0, 3.0]
        },
        {
            "name": "Locality",
            "dtype": "INT",
            "ftype": "continuous",
            "range": [0, 7]
        },
        {
            "name": "Map_Ref",
            "dtype": "INT",
            "ftype": "continuous",
            "range": [1, 13]
        },
        {
            "name": "Latitude",
            "dtype": "INT",
            "ftype": "continuous",
            "range": [1, 11]
        },
        {
            "name": "Altitude",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [100, 300]
        },
        {
            "name": "Rainfall",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [850, 1750]
        },
        {
            "name": "Frosts",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [-2.0, -3.0]
        },
        {
            "name": "Year",
            "dtype": "INT",
            "ftype": "continuous",
            "range": [1800, 1900]
        },
        {
            "name": "Sp",
            "ftype": "categorical",
            "values": ["ag"]
        },
        {
            "name": "PMCno",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1500, 3200]
        },
        {
            "name": "DBH",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [5.0, 50.0]
        },
        {
            "name": "Ht",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [4.0, 13.0]
        },
        {
            "name": "Surv",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [20.0, 100.0]
        },
        {
            "name": "Vig",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [2.0, 5.0]
        },
        {
            "name": "Ins_res",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1.0, 5.0]
        },
        {
            "name": "Stem_Fm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1.0, 5.0]
        },
        {
            "name": "Crown_Fm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1.0, 5.0]
        },
        {
            "name": "Brnch_Fm",
            "dtype": "FLOAT",
            "ftype": "continuous",
            "range": [1.0, 5.0]
        },
        {
            "name": "Utility",
            "dtype": "INT",
            "ftype": "continuous",
            "range": [0, 4]
        }
    ],
    "targets": []
}

In [ ]:
from platiagro.deployment import test_deployment

test_deployment("contract.json")